In [ ]:
import pandas as pd
import sys
sys.path.append("../")
from model import *
from pathlib import Path
from module import * 
import altair as alt
from pytorch_lightning.callbacks.progress import TQDMProgressBar
import torch 
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import argparse
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint
from  pytorch_lightning.loggers.tensorboard import TensorBoardLogger
import h5py

In [ ]:
dataset_type = 'train'
for dataset_type in ["train", "val", "test"]:
    ground_truth = 'normal'
    data_path = f"/kaggle/input/final-hdf5-files/filtered_chunk1_{dataset_type}.hdf5"
    h5_file = h5py.File(data_path, "r")
    keys = list(h5_file.keys())
    person_seq = {}
    print(len(keys), 'comma')

Analyze distance prediction error

Analyze error per person

In [ ]:
backbone = "none" #Originally "resnet"
dataset = "comma"
bs = 1
gpu_num = 1

In [ ]:
def mse_loss(input, target, mask, reduction="mean"):
        out = (input[~mask]-target[~mask])**2
        return out.mean() if reduction == "mean" else out

In [ ]:
def calculate_loss(logits, angle, distance, multitask):
        print(angle, distance, logits)
        if multitask == "multitask":
            logits_angle = torch.tensor([l[0] for l in logits])
            logits_dist = torch.tensor([l[1] for l in logits])
            mask = (distance > 40) | (distance == 0)
            loss_angle = torch.sqrt(mse_loss(logits_angle.squeeze(), angle.squeeze(), mask))
            loss_distance = torch.sqrt(mse_loss(logits_dist.squeeze(), distance.squeeze(), mask))
            loss = loss_angle, loss_distance
            return loss_angle, loss_distance
        elif multitask == "angle":
            mask = distance.squeeze() == 0.0 
            loss = torch.sqrt(mse_loss(logits.squeeze(), angle.squeeze(), mask))
            return loss
        elif multitask == "distance":
            mask = (distance > 40)  | (distance == 0)
            loss = torch.sqrt(mse_loss(logits.squeeze(), angle.squeeze(), mask))
            return loss

In [ ]:
def get_regular_ckpt_from_lightning_checkpoint(state_dict):
    try:
        new_sd = {}
        for k, v in state_dict.items():
            nk = k.replace("model.", "") if k.startswith("model.") else k
            new_sd[nk] = v
        return new_sd
    except Exception:
        return state_dict

In [ ]:
type='distance'

ckpt_root = f"/kaggle/working/ckpts_final_comma_{type}_none_True_1"
#find the latest version -G.R.
versions = glob.glob(os.path.join(ckpt_root, "lightning_logs", "version_*"))
if not versions:
    raise FileNotFoundError("None found")
latest_version = max(versions, key=os.path.getmtime)

# Find checkpoints in the latest version -G.R.
ckpt_files = glob.glob(os.path.join(latest_version, "checkpoints", "*.ckpt"))
if not ckpt_files:
    raise FileNotFoundError("None found.")
checkpoint_path = max(ckpt_files, key=os.path.getmtime)

print(f"Using checkpoint: {checkpoint_path}")

ckpt = torch.load(checkpoint_path)
state_dict = ckpt['state_dict']
state_dict = get_regular_ckpt_from_lightning_checkpoint(state_dict)
#added this to load the model correctly -G.R.
#model.load_state_dict(state_dict)
print('done')

In [ ]:
ckpt_pth_angle = f"/kaggle/working/ckpts_final_comma_angle_none_True_1"
ckpt_pth_distance = f'/kaggle/working/ckpts_final_comma_distance_none_True_1'
ckpt_pth_multitask = f'/kaggle/working/ckpts_final_comma_multitask_none_True_1'

In [ ]:
def save_preds(logits, target, save_name, path):
    df = pd.DataFrame()
    df['logits'] = logits.squeeze().tolist()
    df['target'] = target.squeeze().tolist()
    df.to_csv(f'./{path}/{save_name}.csv', mode='a', index=False, header=False)

In [ ]:
# PASs the prediction csv

p_conceptClip = "/kaggle/working/comma_distance_none_True_100_clip.csv"
p_retinanet = "/kaggle/working/comma_distance_none_True_100_retinanet.csv"
ps = [p_conceptClip,p_retinanet]



In [ ]:
p_concept = "/kaggle/working/comma_distance_none_True_100_clip.csv"
df = pd.read_csv(p_concept, header=None)
df.columns = ['preds', 'targets']

m = (df['targets'] > 40).astype(bool)  | (df['targets'] == 0).astype(bool) 
loss1 = torch.sqrt(mse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m)))
m = (df['targets'] > 50).astype(bool)  | (df['targets'] == 0).astype(bool) 
loss2 = torch.sqrt(mse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m)))
m = (df['targets'] > 60).astype(bool)  | (df['targets'] == 0).astype(bool) 
loss3 = torch.sqrt(mse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m)))
print(loss1.item(), loss2.item(), loss3.item())

Same thing with angle/distance desidered:

In [ ]:

p_concept = '/data1/shared/jessica/data3/data/toyota/ckpts/ckpts_desirednuscenes_distance/lightning_logs/version_6/distance.csv'
p_concept_resnet = '/data1/shared/jessica/data3/data/toyota/ckpts/ckpts_desirednuscenes_distance/lightning_logs/version_7/distance.csv'
p_resnet = '/data1/shared/jessica/data3/data/toyota/ckpts/ckpts_desirednuscenes_distance/lightning_logs/version_9/distance.csv'
ps = [p_concept, p_resnet, p_concept_resnet]
for p in ps: 
    print(p)
    df = pd.read_csv(p, header=None)
    df.columns = ['preds', 'targets']

    m = (df['targets'] > 40).astype(bool)  | (df['targets'] == 0).astype(bool) 
    loss1 = torch.sqrt(mse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m)))
    m = (df['targets'] > 50).astype(bool)  | (df['targets'] == 0).astype(bool) 
    loss2 = torch.sqrt(mse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m)))
    m = (df['targets'] > 60).astype(bool)  | (df['targets'] == 0).astype(bool) 
    loss3 = torch.sqrt(mse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m)))
    print(loss1.item(), loss2.item(), loss3.item())


In [ ]:
loss1.item(), loss2.item(), loss3.item()#dist multi

In [ ]:
loss1.item(), loss2.item(), loss3.item()#dist

In [ ]:
plt.plot(["<40", "<50", "<60"], [loss1.item(), loss2.item(), loss3.item()])
plt.xlabel("Distance")
plt.ylabel("RMSE")

In [ ]:
res = {}

In [ ]:
res

In [ ]:
df = pd.DataFrame()
df['distance'] = res['distance']
df['angle'] = res['angle']
df['driver'] = range(len(res['angle']))
alt.Chart(df).mark_circle(size=60).encode(
    x=alt.X('distance',scale=alt.Scale(domain=(0, 16))),
    y=alt.Y('angle',scale=alt.Scale(domain=(0, 3))),
    tooltip=['driver']
).interactive()


In [ ]:
l = list(zip(*res["multitask"]))
z = l[1]
y = l[0]
df = pd.DataFrame()
df['distance'] = z
df['distance'] = df['distance'].apply(lambda x: x.item())
df['angle'] = y
df['angle'] = df['angle'].apply(lambda x: x.item())
df['driver'] = range(len(y))
alt.Chart(df).mark_circle(size=60).encode(
    x='distance',
    y='angle',
    tooltip=['driver']
).interactive()

In [ ]:
#INSERT THE RIGHT PREDS PATH
preds_path = '/kaggle/working/comma_distance_none_True_100_clip.csv'
#preds_path=f'/data1/jessica/data/toyota/ckpts/ckpts_desiredcomma_distance/lightning_logs/version_1/dist.csv'
df = pd.read_csv(preds_path)
df.columns = ['preds', 'targets']
df = df[df['targets'] != 0]
df['err'] = df['targets'] - df['preds']
ten = df[df['targets'] < 10]
six = df[df['targets'] > 60]
twen = df[(df['targets'] > 10 & (df['targets'] < 20))]
four = df[(df['targets'] > 20 & (df['targets'] < 40))]
fift = df[(df['targets'] > 40 & (df['targets'] < 60))]
ndf = {}
ndf["<10"] = ten['err'].tolist()
ndf["10-20"] = twen['err'].tolist()
ndf["20-40"] = four['err'].tolist()
ndf["40-60"] = fift['err'].tolist()
ndf[">60"] = six['err'].tolist()
my_dict = ndf

fig, ax = plt.subplots()
ax.boxplot(my_dict.values())
ax.set_xticklabels(my_dict.keys())